In [ ]:
import random
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_weights(input_size, hidden_size, output_size):
    return np.random.uniform(size=(input_size, hidden_size), low=-1, high=1), np.random.uniform(size=(hidden_size, output_size), low=-1, high=1)

def forward(x, weights_input_hidden, weights_hidden_output):
    hidden_output = sigmoid(np.dot(x, weights_input_hidden))
    output = sigmoid(np.dot(hidden_output, weights_hidden_output))
    return hidden_output, output

def backward(x, y, output, hidden_output, weights_input_hidden, weights_hidden_output, learning_rate=0.1):
    output_delta = (y - output) * output * (1 - output)
    hidden_layer_delta = output_delta.dot(weights_hidden_output.T) * hidden_output * (1 - hidden_output)

    weights_hidden_output += hidden_output.reshape(-1, 1).dot(output_delta.reshape(1, -1)) * learning_rate
    weights_input_hidden += x.reshape(-1, 1).dot(hidden_layer_delta.reshape(1, -1)) * learning_rate

def train(x_train, y_train, hidden_size, output_size, epochs=1000, learning_rate=0.1):
    input_size = x_train.shape[1]
    weights_input_hidden, weights_hidden_output = initialize_weights(input_size, hidden_size, output_size)

    accuracy_history = []

    for epoch in range(epochs):
        total_loss = 0
        correct_predictions = 0

        for x, y in zip(x_train, y_train):
            hidden_output, output = forward(x, weights_input_hidden, weights_hidden_output)
            loss = np.mean((y - output) ** 2)
            total_loss += loss
            backward(x, y, output, hidden_output, weights_input_hidden, weights_hidden_output, learning_rate)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {total_loss}")

        # Calculate accuracy
        predictions = [predict(x, weights_input_hidden, weights_hidden_output) for x in X_test]
        correct_predictions = np.sum(np.array(predictions) == y_test)
        accuracy = correct_predictions / len(y_test)
        accuracy_history.append(accuracy)

    return weights_input_hidden, weights_hidden_output, accuracy_history

def predict(x, weights_input_hidden, weights_hidden_output):
    _, output = forward(x, weights_input_hidden, weights_hidden_output)
    return 1 if output >= 0.5 else 0

# 데이터 생성 및 전처리
data = load_breast_cancer()
X, y = data.data, data.target
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# MLP 학습
hidden_size = 5
output_size = 1
epochs = 1000
learning_rate = 0.1

weights_input_hidden, weights_hidden_output, accuracy_history = train(X_train, y_train, hidden_size, output_size, epochs, learning_rate)

# 정확도 계산
predictions = [predict(x, weights_input_hidden, weights_hidden_output) for x in X_test]
accuracy = np.sum(np.array(predictions) == y_test) / len(y_test)

print(f"Final Accuracy: {accuracy}")

# 정확도 그래프 그리기
plt.plot(range(epochs), accuracy_history, label="Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()
plt.show()
